In [33]:
import optuna
import numpy as np
import warnings
from scipy.optimize import minimize
import pyscf
from pyscf import gto, scf, dft
from decimal import Decimal
class EnergyGetter():
    def __init__(self, mol2, alpha=0.8, beta=0.2, omega=0.4, basis='cc-pvtz'):
        super(EnergyGetter, self).__init__()
        self.mol2 = mol2
        self.alpha = alpha
        self.beta = beta
        self.omega = omega
        self.xyz = self._mol2_to_xyz()
        self.basis = basis
        
    def _mol2_to_xyz(self):
        xyz_lines = []
        with open(self.mol2, 'r') as f:
            lines = f.readlines()
        atom_section = False
        for line in lines:
            if line.startswith('@<TRIPOS>ATOM'):
                atom_section = True
                continue
            elif line.startswith('@<TRIPOS>'):
                atom_section = False
                continue
            if atom_section and line.strip():
                parts = line.split()
                if len(parts) >= 6: 
                    atom_symbol = parts[1][:2].title()
                    x, y, z = parts[2:5] 
                    xyz_lines.append(f"{atom_symbol} {x} {y} {z}")
        return xyz_lines
        
    @staticmethod
    def create_tunable_b3lyp(xc_alpha=0.19, xc_beta=0.46, xc_omega=0.33, lda_frac=0.3, lyp_frac=0.81, vwn_frac=0.19):
    # Verify exchange part sums to 1
        total_x = xc_alpha + xc_beta + lda_frac
        xc_str = f'RSH({Decimal(str(xc_omega))},{Decimal(str(xc_alpha))},-{Decimal(str(xc_beta))}) + LDA*{Decimal(str(lda_frac))}, LYP*{Decimal(str(lyp_frac))} + VWN*{Decimal(str(vwn_frac))}'
        return xc_str
        
    def calculate_energy(self, charge, spin):
        mol = gto.M(
            atom=self.xyz,
            basis=self.basis,
            charge=charge,
            spin=spin,
        )
        xcf = self.create_tunable_b3lyp(xc_alpha=self.alpha, xc_beta=self.beta, xc_omega=self.omega)
        if charge == 0 and spin == 0:
            mf = scf.RKS(mol)
            mf.xc = xcf
            mf.verbose = 0
            mf.kernel()
            nocc = mol.nelectron // 2
            mo_energy_alpha = mf.mo_energy
            homo = mo_energy_alpha[nocc-1]
            lumo = mo_energy_alpha[nocc]
        else:
            mf = scf.UKS(mol)
            mf.xc = xcf
            mf.verbose = 0
            mf.kernel()
            mo_energy_alpha = mf.mo_energy[0]
            mo_energy_beta = mf.mo_energy[1]
            nocc_alpha = (mol.nelectron + mol.spin) // 2
            nocc_beta = (mol.nelectron - mol.spin) // 2
            
            homo_alpha = mo_energy_alpha[nocc_alpha-1]
            lumo_alpha = mo_energy_alpha[nocc_alpha]
            homo_beta = mo_energy_beta[nocc_beta-1]
            lumo_beta = mo_energy_beta[nocc_beta]
            
            homo = max(homo_alpha, homo_beta)
            lumo = min(lumo_alpha, lumo_beta)
            
        return mf.e_tot, homo, lumo
        
    def get_j_function(self):
        try:
            E_N, E_homo, E_lumo = self.calculate_energy(charge=0, spin=0)
            E_N_plus_1, E_homo_plus_1, E_lumo_plus_1 = self.calculate_energy(charge=-1, spin=1)
            E_N_minus_1, E_homo_minus_1, E_lumo_minus_1 = self.calculate_energy(charge=1, spin=1)
            J_N = abs(E_homo + E_N_minus_1 - E_N)
            J_N_plus_1 = abs(E_homo_plus_1 + E_N - E_N_plus_1)
            J = J_N**2 + J_N_plus_1**2
            return J
        except Exception as e:
            warnings.warn(f"计算J函数失败: {e}")
            return float('inf')
    
    def optimize_with_optuna(self, n_trials=100, study_name=None, storage=None):
        """
        使用Optuna优化参数
        """
        def objective(trial):
            # 建议参数值
            alpha = trial.suggest_float('alpha', 0.0, 1.0)
            beta = trial.suggest_float('beta', 0.0, 1.0)
            omega = trial.suggest_float('omega', 0,1.0)
            
            # 约束条件: alpha + beta <= 1
            if alpha + beta > 1.0:
                return float('inf')
            
            # 更新参数
            self.alpha = alpha
            self.beta = beta
            self.omega = omega
            
            # 计算目标函数
            J = self.get_j_function()
            
            # 记录参数和结果
            trial.set_user_attr('alpha', alpha)
            trial.set_user_attr('beta', beta)
            trial.set_user_attr('omega', omega)
            
            return J
        
        # 创建或加载研究
        if study_name and storage:
            study = optuna.create_study(
                study_name=study_name,
                storage=storage,
                load_if_exists=True,
                direction='minimize'
            )
        else:
            study = optuna.create_study(direction='minimize')
        
        # 执行优化
        study.optimize(objective, n_trials=n_trials)
        
        # 获取最佳参数
        best_params = study.best_params
        best_value = study.best_value
        
        # 更新实例参数
        self.alpha = best_params['alpha']
        self.beta = best_params['beta']
        self.omega = best_params['omega']
        
        return study, best_params, best_value
    
    def optimize_with_optuna_advanced(self, n_trials=100, timeout=None, 
                                    pruner=optuna.pruners.HyperbandPruner(),
                                    sampler=optuna.samplers.TPESampler()):
        """
        高级Optuna优化，包含更多配置选项
        """
        def objective(trial):
            # 使用不同的建议策略
            alpha = trial.suggest_float('alpha', 0.0, 1.0)
            beta = trial.suggest_float('beta', 0.0, 1.0)
            omega = trial.suggest_float('omega', 0.0, 1.0)
            
            # 约束条件
            if alpha + beta > 1.0:
                return float('inf')
            
            # 更新参数
            self.alpha = alpha
            self.beta = beta
            self.omega = omega
            
            # 计算目标函数
            J = self.get_j_function()
            
            # 如果J是无穷大，提前终止这个试验
            if np.isinf(J):
                raise optuna.TrialPruned()
            
            return J
        
        # 创建研究
        study = optuna.create_study(
            direction='minimize',
            sampler=sampler,
            pruner=pruner
        )
        
        # 执行优化
        study.optimize(objective, n_trials=n_trials, timeout=timeout)
        
        # 获取最佳参数
        best_params = study.best_params
        best_value = study.best_value
        
        # 更新实例参数
        self.alpha = best_params['alpha']
        self.beta = best_params['beta']
        self.omega = best_params['omega']
        
        return study, best_params, best_value
    
    def visualize_optimization(self, study):
        """
        可视化优化结果
        """
        try:
            # 参数重要性
            fig1 = optuna.visualization.plot_param_importances(study)
            fig1.show()
            
            # 优化历史
            fig2 = optuna.visualization.plot_optimization_history(study)
            fig2.show()
            
            # 参数关系
            fig3 = optuna.visualization.plot_parallel_coordinate(study)
            fig3.show()
            
        except Exception as e:
            print(f"可视化失败: {e}")


# 使用示例
if __name__ == '__main__':
    mol2 = '/Users/jiaoyuan/Documents/GitHub/ADOPTXC/module/net.mol2'
    energy_getter = EnergyGetter(mol2)
    
    print("开始Optuna优化...")
    
    # 基本优化
    study, best_params, best_value = energy_getter.optimize_with_optuna(
        n_trials=100  # 可以根据需要调整试验次数
    )
    
    print(f"优化完成!")
    print(f"最佳参数: alpha={best_params['alpha']:.4f}, beta={best_params['beta']:.4f}, omega={best_params['omega']:.4f}")
    print(f"最小J值: {best_value:.6f}")
    
    # 可视化结果
    energy_getter.visualize_optimization(study)
    
    # 高级优化示例 (取消注释以使用)
   
    print("开始高级Optuna优化...")
    study_advanced, best_params_advanced, best_value_advanced = energy_getter.optimize_with_optuna_advanced(
        n_trials=100,
        timeout=3600,  # 1小时超时
        pruner=optuna.pruners.HyperbandPruner(),
        sampler=optuna.samplers.TPESampler(seed=42)
    )
    energy_getter.visualize_optimization(study_advanced)
    print(f"高级优化完成!")
    print(f"最佳参数: alpha={best_params_advanced['alpha']:.4f}, beta={best_params_advanced['beta']:.4f}, omega={best_params_advanced['omega']:.4f}")
    print(f"最小J值: {best_value_advanced:.6f}")


[I 2025-11-01 10:06:39,116] A new study created in memory with name: no-name-946ef0e6-c67c-419f-ae9f-559d7a424c4e
[I 2025-11-01 10:06:39,117] Trial 0 finished with value: inf and parameters: {'alpha': 0.637920775329549, 'beta': 0.36410044864073887, 'omega': 0.1546782207250048}. Best is trial 0 with value: inf.
[I 2025-11-01 10:06:39,117] Trial 1 finished with value: inf and parameters: {'alpha': 0.6137595144914252, 'beta': 0.8918670017157789, 'omega': 0.8192565938551006}. Best is trial 0 with value: inf.
[I 2025-11-01 10:06:39,118] Trial 2 finished with value: inf and parameters: {'alpha': 0.6729865503504725, 'beta': 0.41131368664580736, 'omega': 0.5208933943367582}. Best is trial 0 with value: inf.
[I 2025-11-01 10:06:39,118] Trial 3 finished with value: inf and parameters: {'alpha': 0.6508106244803975, 'beta': 0.4400626446473622, 'omega': 0.3803401556422995}. Best is trial 0 with value: inf.
[I 2025-11-01 10:06:39,119] Trial 4 finished with value: inf and parameters: {'alpha': 0.4124

开始Optuna优化...


[I 2025-11-01 10:06:43,572] Trial 7 finished with value: 0.13241763581255955 and parameters: {'alpha': 0.047516917003024406, 'beta': 0.7442124754020605, 'omega': 0.9321960520019236}. Best is trial 7 with value: 0.13241763581255955.
[I 2025-11-01 10:06:45,625] Trial 8 finished with value: 0.07152278301612246 and parameters: {'alpha': 0.0490220697805166, 'beta': 0.10359710396497868, 'omega': 0.2326210755430903}. Best is trial 8 with value: 0.07152278301612246.
[I 2025-11-01 10:06:47,452] Trial 9 finished with value: 0.051766722288145925 and parameters: {'alpha': 0.129923480484888, 'beta': 0.09020011232755876, 'omega': 0.8811890098376313}. Best is trial 9 with value: 0.051766722288145925.
[I 2025-11-01 10:06:49,150] Trial 10 finished with value: 0.039715171376684746 and parameters: {'alpha': 0.20170750922073655, 'beta': 0.005477731947336542, 'omega': 0.715107378123494}. Best is trial 10 with value: 0.039715171376684746.
[I 2025-11-01 10:06:50,847] Trial 11 finished with value: 0.036071499

优化完成!
最佳参数: alpha=0.8005, beta=0.1677, omega=0.3533
最小J值: 0.000418


[W 2025-11-01 10:08:52,119] Trial 0 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:08:52,119] Trial 1 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:08:52,119] Trial 2 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:08:52,119] Trial 3 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:08:52,120] Trial 4 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:08:52,120] Trial 5 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:08:52,120] Trial 6 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:08:52,120] Trial 17 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:08:52,120] Trial 20 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:08:52,121] Trial 2

[I 2025-11-01 10:08:52,128] A new study created in memory with name: no-name-e7559a00-6897-4ea1-8516-2a0349ec33ce
[I 2025-11-01 10:08:52,129] Trial 0 finished with value: inf and parameters: {'alpha': 0.3745401188473625, 'beta': 0.9507143064099162, 'omega': 0.7319939418114051}. Best is trial 0 with value: inf.


开始高级Optuna优化...


[I 2025-11-01 10:08:53,989] Trial 1 finished with value: 0.009467451627341427 and parameters: {'alpha': 0.5986584841970366, 'beta': 0.15601864044243652, 'omega': 0.15599452033620265}. Best is trial 1 with value: 0.009467451627341427.
[I 2025-11-01 10:08:58,622] Trial 2 finished with value: 0.15829501661317782 and parameters: {'alpha': 0.05808361216819946, 'beta': 0.8661761457749352, 'omega': 0.6011150117432088}. Best is trial 1 with value: 0.009467451627341427.
[I 2025-11-01 10:09:00,641] Trial 3 finished with value: 0.0008432018528075685 and parameters: {'alpha': 0.7080725777960455, 'beta': 0.020584494295802447, 'omega': 0.9699098521619943}. Best is trial 3 with value: 0.0008432018528075685.
[I 2025-11-01 10:09:00,641] Trial 4 finished with value: inf and parameters: {'alpha': 0.8324426408004217, 'beta': 0.21233911067827616, 'omega': 0.18182496720710062}. Best is trial 3 with value: 0.0008432018528075685.
[I 2025-11-01 10:09:02,541] Trial 5 finished with value: 0.05442583588832183 and

[W 2025-11-01 10:11:00,139] Trial 0 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:11:00,139] Trial 4 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:11:00,139] Trial 8 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:11:00,139] Trial 9 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:11:00,140] Trial 13 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:11:00,140] Trial 14 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:11:00,140] Trial 15 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:11:00,140] Trial 16 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:11:00,141] Trial 25 is omitted in visualization because its objective value is inf or nan.
[W 2025-11-01 10:11:00,141] Tria

高级优化完成!
最佳参数: alpha=0.7731, beta=0.2124, omega=0.8131
最小J值: 0.000477


In [41]:
import pyscf
from pyscf import gto, scf, dft
from decimal import Decimal
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

class EnergyGetter():
    def __init__(self, mol2, alpha=0.8, beta=0.2, omega=0.4, basis='sto-3g'):
        super(EnergyGetter, self).__init__()
        self.mol2 = mol2
        self.alpha = alpha
        self.beta = beta
        self.omega = omega
        self.xyz = self._mol2_to_xyz()
        self.basis = basis
        
    def _mol2_to_xyz(self):
        xyz_lines = []
        with open(self.mol2, 'r') as f:
            lines = f.readlines()
        atom_section = False
        for line in lines:
            if line.startswith('@<TRIPOS>ATOM'):
                atom_section = True
                continue
            elif line.startswith('@<TRIPOS>'):
                atom_section = False
                continue
            if atom_section and line.strip():
                parts = line.split()
                if len(parts) >= 6: 
                    atom_symbol = parts[1][:2].title()
                    x, y, z = parts[2:5] 
                    xyz_lines.append(f"{atom_symbol} {x} {y} {z}")
        return xyz_lines
        
    @staticmethod
    def create_tunable_b3lyp(xc_alpha=0.2, xc_beta=0.72, xc_omega=None, lda_frac=0.08, lyp_frac=0.81, vwn_frac=0.19):
    # Verify exchange part sums to 1
        total_x = xc_alpha + xc_beta + lda_frac
        if xc_omega is None:
        # Standard B3LYP form, no range separation
            xc_str = f'HF*{Decimal(str(xc_alpha))} + LDA*{Decimal(str(lda_frac))} + B88*{Decimal(str(xc_beta))}, LYP*{Decimal(str(lyp_frac))} + VWN*{Decimal(str(vwn_frac))}'
        else:
        # Range separated version
            xc_str = f'RSH({Decimal(str(xc_omega))},{Decimal(str(xc_alpha))},-{Decimal(str(xc_beta))}) + LDA*{Decimal(str(lda_frac))}, LYP*{Decimal(str(lyp_frac))} + VWN*{Decimal(str(vwn_frac))}'
        return xc_str
        
    def calculate_energy(self, charge, spin):
        mol = gto.M(
            atom=self.xyz,
            basis=self.basis,
            charge=charge,
            spin=spin,
        )
        xcf = self.create_tunable_b3lyp(xc_alpha=self.alpha, xc_beta=self.beta, xc_omega=self.omega)
        if charge == 0 and spin == 0:
            mf = scf.RKS(mol)
            mf.xc = xcf
            mf.verbose = 0
            mf.kernel()
            nocc = mol.nelectron // 2
            mo_energy_alpha = mf.mo_energy
            homo = mo_energy_alpha[nocc-1]
            lumo = mo_energy_alpha[nocc]
        else:
            mf = scf.UKS(mol)
            mf.xc = xcf
            mf.verbose = 0
            mf.kernel()
            mo_energy_alpha = mf.mo_energy[0]
            mo_energy_beta = mf.mo_energy[1]
            nocc_alpha = (mol.nelectron + mol.spin) // 2
            nocc_beta = (mol.nelectron - mol.spin) // 2
            
            homo_alpha = mo_energy_alpha[nocc_alpha-1]
            lumo_alpha = mo_energy_alpha[nocc_alpha]
            homo_beta = mo_energy_beta[nocc_beta-1]
            lumo_beta = mo_energy_beta[nocc_beta]
            
            homo = max(homo_alpha, homo_beta)
            lumo = min(lumo_alpha, lumo_beta)
            
        return mf.e_tot, homo, lumo
        
    def get_j_function(self):
        E_N, E_homo, E_lumo = self.calculate_energy(charge=0, spin=0)
        E_N_plus_1, E_homo_plus_1, E_lumo_plus_1 = self.calculate_energy(charge=-1, spin=1)
        E_N_minus_1, E_homo_minus_1, E_lumo_minus_1 = self.calculate_energy(charge=1, spin=1)
        J_N = abs(E_homo + E_N_minus_1 - E_N)
        J_N_plus_1 = abs(E_homo_plus_1 + E_N - E_N_plus_1)
        J = J_N**2 + J_N_plus_1**2
        return J
        
    def forward(self):
        J = self.get_j_function()
        alpha = self.alpha 
        beta = self.beta
        omega = self.omega
        return J, alpha, beta, omega


class ParameterScanner:
    def __init__(self, mol2_file, basis='3-21g'):
        self.mol2_file = mol2_file
        self.basis = basis
        
    def scan_parameters(self, alpha_range, beta_range, omega_range):
        """
        对三个参数进行扫描
        
        Parameters:
        -----------
        alpha_range : array-like
            alpha参数的扫描范围
        beta_range : array-like
            beta参数的扫描范围  
        omega_range : array-like
            omega参数的扫描范围
            
        Returns:
        --------
        results : pandas.DataFrame
            包含所有参数组合和对应J值的DataFrame
        """
        results = []
        
        # 创建参数网格
        total_combinations = len(alpha_range) * len(beta_range) * len(omega_range)
        
        print(f"开始参数扫描，总共 {total_combinations} 个组合...")
        
        with tqdm(total=total_combinations) as pbar:
            for alpha in alpha_range:
                for beta in beta_range:
                    for omega in omega_range:
                        try:
                            # 创建EnergyGetter实例
                            energy_getter = EnergyGetter(
                                mol2=self.mol2_file,
                                alpha=alpha,
                                beta=beta,
                                omega=omega,
                                basis=self.basis
                            )
                            
                            # 计算J函数
                            J = energy_getter.get_j_function()
                            
                            # 保存结果
                            results.append({
                                'alpha': alpha,
                                'beta': beta,
                                'omega': omega,
                                'J_value': J
                            })
                            
                        except Exception as e:
                            print(f"参数组合 (alpha={alpha}, beta={beta}, omega={omega}) 计算失败: {e}")
                            results.append({
                                'alpha': alpha,
                                'beta': beta,
                                'omega': omega,
                                'J_value': np.nan
                            })
                        
                        pbar.update(1)
        
        return pd.DataFrame(results)
    
    def plot_results(self, results_df, fixed_param=None, fixed_value=None):

        if fixed_param and fixed_value is not None:
        # 固定一个参数，显示另外两个参数的热图
            fixed_df = results_df[results_df[fixed_param] == fixed_value]
        
            if len(fixed_df) > 0:
            # 确定另外两个参数
                params = ['alpha', 'beta', 'omega']
                params.remove(fixed_param)
                param1, param2 = params
            
            # 创建热图数据
                pivot_data = fixed_df.pivot_table(
                values='J_value', 
                index=param1, 
                columns=param2,
                aggfunc='mean'
            )
            
                plt.figure(figsize=(10, 8))
                plt.imshow(pivot_data.values, cmap='viridis', aspect='auto',
                      extent=[fixed_df[param2].min(), fixed_df[param2].max(),
                             fixed_df[param1].min(), fixed_df[param1].max()])
                plt.colorbar(label='J Value')
                plt.xlabel(param2)
                plt.ylabel(param1)
                plt.title(f'J Values (fixed {fixed_param} = {fixed_value})')
                plt.show()
            else:
                print(f"没有找到 {fixed_param} = {fixed_value} 的数据")
        else:
        # 创建三个子图，每个都使用omega值作为颜色
            fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # 第一个图: alpha vs beta, 颜色表示omega
            scatter1 = axes[0].scatter(results_df['alpha'], results_df['beta'], 
                                  c=results_df['omega'], cmap='viridis', 
                                  alpha=0.7, s=50)
            axes[0].set_xlabel('Alpha')
            axes[0].set_ylabel('Beta')
            axes[0].set_title('Alpha vs Beta (颜色表示Omega)')
            plt.colorbar(scatter1, ax=axes[0], label='Omega Value')
        
        # 第二个图: alpha vs J_value, 颜色表示omega
            scatter2 = axes[1].scatter(results_df['alpha'], results_df['J_value'], 
                                  c=results_df['omega'], cmap='plasma', 
                                  alpha=0.7, s=50)
            axes[1].set_xlabel('Alpha')
            axes[1].set_ylabel('J Value')
            axes[1].set_title('Alpha vs J Value (颜色表示Omega)')
            plt.colorbar(scatter2, ax=axes[1], label='Omega Value')
        
        # 第三个图: beta vs J_value, 颜色表示omega
            scatter3 = axes[2].scatter(results_df['beta'], results_df['J_value'], 
                                  c=results_df['omega'], cmap='plasma', 
                                  alpha=0.7, s=50)
            axes[2].set_xlabel('Beta')
            axes[2].set_ylabel('J Value')
            axes[2].set_title('Beta vs J Value (颜色表示Omega)')
            plt.colorbar(scatter3, ax=axes[2], label='Omega Value')
        
            plt.tight_layout()
            plt.show()
        
        # 额外创建一个3D散点图来展示三个参数的关系
            fig = plt.figure(figsize=(10, 8))
            ax = fig.add_subplot(111, projection='3d')
        
            scatter = ax.scatter(results_df['alpha'], results_df['beta'], results_df['J_value'],
                           c=results_df['omega'], cmap='viridis', s=50, alpha=0.7)
            ax.set_xlabel('Alpha')
            ax.set_ylabel('Beta')
            ax.set_zlabel('J Value')
            ax.set_title('3D参数空间 (颜色表示Omega)')
            plt.colorbar(scatter, ax=ax, label='Omega Value')
            plt.show()

def main():
    mol2_file = '/Users/jiaoyuan/Documents/GitHub/ADOPTXC/module/net.mol2'
    
    # 创建参数扫描器
    scanner = ParameterScanner(mol2_file)
    
    # 定义参数扫描范围
    # 可以根据需要调整范围和步长
    alpha_range = np.linspace(0.01, 0.9, 20)  # 20个点
    beta_range = np.linspace(0.01, 0.9, 20)   # 20个点  
    omega_range = np.linspace(0.05, 0.5, 10)  # 20个点
    
    print("参数扫描范围:")
    print(f"Alpha: {alpha_range}")
    print(f"Beta: {beta_range}") 
    print(f"Omega: {omega_range}")
    
    # 执行扫描
    results = scanner.scan_parameters(alpha_range, beta_range, omega_range)
    
    # 保存结果到CSV文件
    results.to_csv('parameter_scan_results.csv', index=False)
    print("结果已保存到 parameter_scan_results.csv")
    
    # 显示最佳参数组合
    best_result = results.loc[results['J_value'].idxmin()]
    print("\n最佳参数组合:")
    print(f"Alpha: {best_result['alpha']:.3f}")
    print(f"Beta: {best_result['beta']:.3f}")
    print(f"Omega: {best_result['omega']:.3f}")
    print(f"J值: {best_result['J_value']:.6f}")
    
    # 可视化结果
    scanner.plot_results(results)
    
    # 固定一个参数查看热图
    scanner.plot_results(results, fixed_param='omega', fixed_value=0.05)


if __name__ == '__main__':
    main()

参数扫描范围:
Alpha: [0.01       0.05684211 0.10368421 0.15052632 0.19736842 0.24421053
 0.29105263 0.33789474 0.38473684 0.43157895 0.47842105 0.52526316
 0.57210526 0.61894737 0.66578947 0.71263158 0.75947368 0.80631579
 0.85315789 0.9       ]
Beta: [0.01       0.05684211 0.10368421 0.15052632 0.19736842 0.24421053
 0.29105263 0.33789474 0.38473684 0.43157895 0.47842105 0.52526316
 0.57210526 0.61894737 0.66578947 0.71263158 0.75947368 0.80631579
 0.85315789 0.9       ]
Omega: [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
开始参数扫描，总共 4000 个组合...


  0%|          | 1/4000 [30:18<2019:51:16, 1818.32s/it]


KeyboardInterrupt: 